In [3]:
import pandas as pd
import numpy as np

Read in data

In [4]:
HIEdata = pd.read_csv("_SQL_query_Combined_Jan2019_Jan2021.csv")
HIEdata.head()

/home/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,patient_id,age,gender,race,ethnicity,language,encounter_id,encounter_class,admit_type,facility,...,financial_class,vip_indicator,encounter_category,encounter_type,encounter_description,encounter_code,encounter_code_method,encounter_priority,file_number,tract_fips
0,284850,120,M,NaN,NaN,NaN,4560864,EMERGENCY,Emergency,New Hanover Regional Medical Center,...,Self-pay,NaN,Diagnosis,ADMIT,GSW to belly,NaN,Free Text,NaN,6.0,37129011500
1,1133656,21,M,White,Not Hispanic or Latino,en,21341790,OUTPATIENT,NaN,CCHIE Practices,...,NaN,NaN,Procedure,NaN,NO SHOW,NOSHOW,CPT,0.0,17.0,37133002800
2,132502,73,M,White,Not Hispanic or Latino,en,20792679,OUTPATIENT,NaN,Wilmington Health Practices,...,NaN,NaN,Procedure,NaN,SUBSEQUENT HOSPITAL CARE,99233,CPT,0.0,4.0,37129011705
3,132502,73,M,White,Not Hispanic or Latino,en,20792679,OUTPATIENT,NaN,Wilmington Health Practices,...,NaN,NaN,Procedure,NaN,HOSPITAL DISCHARGE DAY,99239,CPT,0.0,4.0,37129011705
4,239721,99,M,White,Not Hispanic or Latino,en,18619927,OUTPATIENT,NaN,Wilmington Health Practices,...,NaN,NaN,Procedure,NaN,OBSERVATION CARE DISCHARGE,99217,CPT,0.0,21.0,37129012008


Copy Data

In [1]:
Overweight = HIEdata.copy()

NameError: name 'HIEdata' is not defined

Remove '.' from encounter code column

In [66]:
Overweight['encounter_code'] = Overweight['encounter_code'].str.replace('.','', regex=True)

Fill NA values in encounter_code with 'Not Listed'

In [68]:
Overweight['encounter_code'] = Overweight['encounter_code'].fillna('Not Listed')

Create HistoryOfObesity_yn column where value is 'Yes' if encounter_code starts with E66, Z6854, or Z6853 and value is 'No' otherwise

In [74]:
ObesityCode = ['E66', 'Z6854', 'Z6853']
Overweight["HistoryOfObesity_yn"] = pd.np.where(Overweight.encounter_code.str.startswith("E66"),'Yes',
                                    pd.np.where(Overweight.encounter_code.str.startswith("Z6854"),'Yes',
                                    pd.np.where(Overweight.encounter_code.str.startswith("Z6853"),'Yes', 'No')))   

<ipython-input-74-ab26c5bbe7cb>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  Overweight["HistoryOfObesity_yn"] = pd.np.where(Overweight.encounter_code.str.startswith("E66"),'Yes',
<ipython-input-74-ab26c5bbe7cb>:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  pd.np.where(Overweight.encounter_code.str.startswith("Z6854"),'Yes',
<ipython-input-74-ab26c5bbe7cb>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  pd.np.where(Overweight.encounter_code.str.startswith("Z6853"),'Yes', 'No')))


Subset on only patients with a history of obesity

In [75]:
ObesityHistoryYes = Overweight[Overweight['HistoryOfObesity_yn']=='Yes']

Make a list of unique patient ID's that have a history of obesity using the subsetted created above

In [85]:
ObesePatientId = (ObesityHistoryYes['patient_id'].unique()).tolist()

Add a HistoryOfObesity_yn column to the original dataset where value is 'Yes' for all patient id's in the ObesePatientID (patients with a history of obesity) and 'No' otherwise

In [89]:
HIEdata['HistoryOfObesity_yn'] = np.where(HIEdata['patient_id'].isin(ObesePatientId),'Yes','No')

Check value counts

In [93]:
HIEdata.HistoryOfObesity_yn.value_counts()

No     7792930
Yes    2043081
Name: HistoryOfObesity_yn, dtype: int64

Write data to csv

In [94]:
HIEdata.to_csv('HIEdata_HistoryOfObesity.csv')